In [ ]:
!pip install -q gradio requests

In [ ]:
import os

# 🔑 ВСТАВЬ СЮДА СВОЙ КЛЮЧ
os.environ["PERPLEXITY_API_KEY"] = "ppplx-dddXFbFD7qguGd9T8LVnrauz98765mYNCBHq091vZaZ33xtwtfyODOD"

In [ ]:
import os
import re
import requests
import gradio as gr

# =========================
# CONFIG
# =========================
PPLX_API_KEY = os.environ.get("PERPLEXITY_API_KEY", "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")
PPLX_BASE_URL = "https://api.perplexity.ai"
MODEL_NAME = "sonar-pro"
TIMEOUT = 40


# =========================
# SYSTEM PROMPT (RU ONLY)
# =========================
SYSTEM_PROMPT = """
Ты — AI-репетитор по техническим дисциплинам:
машинное обучение, глубокое обучение, NLP, LLM и диффузионные модели.

Ты отвечаешь СТРОГО на русском языке.

Формат ответа ОБЯЗАТЕЛЕН:

1) Кратко
2) Пояснение
3) Пример (код или псевдокод)
4) Источники (Markdown-ссылки)
---
5) Самопроверка (3–5 вопросов)
6) Рекомендуемые темы (маркированный список)

Правила:
- Используй ТОЛЬКО переданные источники.
- Если данных недостаточно — напиши об этом явно.
- Предпочитай NeurIPS, ICML, ICLR, ACL, CVPR, arXiv.
- В блоке «Рекомендуемые темы» используй ТОЛЬКО формат:
  - Название темы
"""


# =========================
# FALLBACK TOPICS (RU)
# =========================
TOPIC_HIERARCHY = {
    "Machine Learning": {
        "Beginner": [
            "Что такое машинное обучение",
            "Обучение с учителем и без учителя",
            "Переобучение и регуляризация",
            "Функции потерь",
            "Градиентный спуск",
            "Метрики качества моделей",
        ],
        "Intermediate": [
            "Градиентный бустинг",
            "Bias-Variance Tradeoff",
            "Кросс-валидация",
            "Подбор гиперпараметров",
            "Интерпретация моделей (SHAP)",
        ],
        "Advanced": [
            "Причинно-следственный вывод",
            "Сдвиг распределений",
            "Мониторинг ML-моделей",
            "Этика и fairness в ML",
        ],
    },

    "Large Language Models": {
        "Beginner": [
            "Что такое Transformer",
            "Self-attention механизм",
            "Токены и контекст",
            "Prompt engineering",
            "Почему LLM галлюцинируют",
        ],
        "Intermediate": [
            "Архитектура Transformer",
            "Instruction tuning",
            "RAG (Retrieval-Augmented Generation)",
            "Оценка качества LLM",
            "Prompt injection",
        ],
        "Advanced": [
            "RLHF и DPO",
            "Агентные LLM-системы",
            "Безопасность LLM",
            "Редактирование знаний моделей",
        ],
    },
}


# =========================
# HELPERS
# =========================
def default_question(area, level):
    if area == "Machine Learning" and level == "Beginner":
        return "Что такое машинное обучение? Объясни простыми словами."
    if area == "Large Language Models":
        return "Объясни архитектуру Transformer и механизм attention."
    return f"Объясни ключевую тему уровня {level} в области {area}."


def extract_next_topics(text):
    if not text:
        return []

    match = re.search(r"Рекомендуемые темы\s*[:：]?\n", text, re.IGNORECASE)
    if not match:
        return []

    block = text[match.end():]
    topics = []

    for line in block.split("\n"):
        m = re.match(r"^[-•*]\s+(.*)", line.strip())
        if m:
            topics.append(m.group(1).strip())

    return list(dict.fromkeys(topics))[:8]


def fallback_topics(area, level):
    return TOPIC_HIERARCHY.get(area, {}).get(level, [])


# =========================
# PERPLEXITY API
# =========================
def pplx_search(query):
    url = f"{PPLX_BASE_URL}/search"
    headers = {
        "Authorization": f"Bearer {PPLX_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {"query": query, "max_results": 5}

    try:
        r = requests.post(url, headers=headers, json=payload, timeout=TIMEOUT)
        return r.json().get("results", [])
    except:
        return []


def pplx_chat(question, area, level, sources):
    url = f"{PPLX_BASE_URL}/chat/completions"
    headers = {
        "Authorization": f"Bearer {PPLX_API_KEY}",
        "Content-Type": "application/json",
    }

    user_content = f"""
ОБЛАСТЬ: {area}
УРОВЕНЬ: {level}

ВОПРОС:
{question}

ИСТОЧНИКИ:
{sources}
"""

    payload = {
        "model": MODEL_NAME,
        "temperature": 0.2,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_content},
        ],
    }

    r = requests.post(url, headers=headers, json=payload, timeout=TIMEOUT)
    return r.json()["choices"][0]["message"]["content"]


# =========================
# BACKEND LOGIC
# =========================
def handle_question(area, level, question):
    if not question.strip():
        question = default_question(area, level)

    sources = pplx_search(question)
    sources_text = "\n".join(
        [f"- {s.get('title','')} ({s.get('url','')})" for s in sources]
    )

    try:
        answer = pplx_chat(question, area, level, sources_text)
    except:
        answer = "❌ Ошибка API. Попробуйте позже."

    next_topics = extract_next_topics(answer)
    if not next_topics:
        next_topics = fallback_topics(area, level)

    return answer, gr.Dropdown(choices=next_topics)


def handle_followup(area, level, topic):
    return handle_question(area, level, topic)


# =========================
# GRADIO UI (COLAB)
# =========================
with gr.Blocks() as demo:
    gr.Markdown("## 🎓 AI-Репетитор по ML / DL / NLP / LLM / Diffusion")

    with gr.Row():
        area = gr.Dropdown(
            ["Machine Learning", "Deep Learning", "NLP", "Large Language Models", "Diffusion Models"],
            label="Область"
        )
        level = gr.Radio(
            ["Beginner", "Intermediate", "Advanced"],
            label="Уровень"
        )

    question = gr.Textbox(
        label="Ваш вопрос или тема",
        placeholder="Например: Объясни архитектуру Transformer"
    )

    ask_btn = gr.Button("Задать вопрос")
    answer_md = gr.Markdown()

    next_topics = gr.Dropdown(label="📌  Ответы ", interactive=True)
    continue_btn = gr.Button("Продолжить обучение")

    ask_btn.click(
        handle_question,
        inputs=[area, level, question],
        outputs=[answer_md, next_topics],
    )

    continue_btn.click(
        handle_followup,
        inputs=[area, level, next_topics],
        outputs=[answer_md, next_topics],
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bb3f8e2f977c756719.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
